<a href="https://colab.research.google.com/github/masadeghi/dementia_clf_fmri/blob/main/experiments/analysis_scripts/fmri_dementia_resnet_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import dependencies

In [ ]:
import os
import time
import copy
import math
from pprint import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, balanced_accuracy_score
from sklearn.utils import class_weight

!pip install -qU focal_loss_torch
from focal_loss.focal_loss import FocalLoss

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
!pip install -qU timm
import timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.9 MB/s eta 0:00:00


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import data

In [ ]:
# Mount Google drive
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data_path = "/content/gdrive/MyDrive/Coding projects/resting_fmri_dementia"

labels = pd.read_csv(os.path.join(data_path, 'labels_new.csv'))

## Train/val/test split

In [ ]:
# Splitting out the test set
labels_train_val, labels_test = train_test_split(labels, test_size = 0.1,
                                            shuffle = True, random_state = 42,
                                            stratify = labels['label'])

# Train/val split
labels_train, labels_val = train_test_split(labels_train_val, test_size = 0.2,
                                            shuffle = True, random_state = 42,
                                            stratify = labels_train_val['label'])

In [ ]:
labels_train_val = labels_train_val.reset_index()
labels_train = labels_train.reset_index()
labels_val = labels_val.reset_index()
labels_test = labels_test.reset_index()

In [ ]:
# labels_train_val.to_csv(os.path.join(data_path, 'labels_train_val.csv'), header = True)
# labels_test.to_csv(os.path.join(data_path, 'labels_test.csv'), header = True)
# labels_train.to_csv(os.path.join(data_path, 'labels_train.csv'), header = True)
# labels_val.to_csv(os.path.join(data_path, 'labels_val.csv'), header = True)

## Standardizing the data
* Get the means and standard deviations across all of the training set fMRIs
* Standardize the whole dataset using the calculated means and standard deviations

In [ ]:
# # Getting all of the training data

# def get_datasets(data_csv):

#   X = np.empty((len(data_csv), 200, 140))
#   y = np.empty(len(data_csv))

#   for i, row in data_csv.iterrows():
#     matrix_name = row['matrix']
#     matrix_label = row['label']
#     matrix_path = os.path.join(data_path, 'data_dir', matrix_name)
#     matrix_arr = np.genfromtxt(matrix_path, delimiter = ',')
#     matrix_arr = matrix_arr[:200, :140]
#     X[i] = matrix_arr
#     y[i] = matrix_label

#   return X, y

In [ ]:
# X_train, y_train = get_datasets(labels_train)

In [ ]:
# train_mean = np.mean(X_train)
# train_std = np.std(X_train)
# train_mean, train_std

In [ ]:
# os.makedirs(os.path.join(data_path, 'scaled_data_dir'))

In [ ]:
# save_path = "/content/gdrive/MyDrive/Coding projects/resting_fmri_dementia/scaled_data_dir"

# for i, row in labels.iterrows():
#   matrix_name = row['matrix']
#   matrix_path = os.path.join(data_path, 'data_dir', matrix_name)
#   matrix_arr = np.genfromtxt(matrix_path, delimiter = ',')
#   matrix_arr_scaled = (matrix_arr - train_mean)/train_std
#   new_path = os.path.join(save_path, matrix_name)
#   np.savetxt(os.path.join(save_path, matrix_name), matrix_arr_scaled, delimiter = ',')

# Setup Weights and Biases

In [ ]:
!pip install -qU wandb
import wandb

wandb.login(key = '####')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 KB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.0 MB/s eta 0:00:00


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Setting up the weights and biases sweep

sweep_config = {
    'method' : 'random'
}

metric = {
    'name': 'val_f1',
    'goal': 'maximize'
}

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {'values': ['adam', 'sgd']},
    'fc_layer_size': {'values': [128, 256, 512]},
    'dropout': {'values': [0, 0.2, 0.4, 0.6]},
    'epochs': {'value': 8},
    'learning_rate': {'distribution': 'uniform', 'min': 0.0001, 'max': 0.01},
    'train_batch_size': {'distribution': 'q_log_uniform_values', 'q': 1, 'min': 32, 'max': 128},
    'val_batch_size': {'distribution': 'q_log_uniform_values', 'q': 2, 'min': 32, 'max': 256},
    'momentum': {'distribution': 'uniform', 'min' : 0.7, 'max': 1},
    'architecture': {'value': 'resnet18'},
    'gamma': {'values': [0.5, 2 , 4]}
}

sweep_config['parameters'] = parameters_dict

pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'val_f1'},
 'parameters': {'architecture': {'value': 'resnet18'},
                'dropout': {'values': [0, 0.2, 0.4, 0.6]},
                'epochs': {'value': 8},
                'fc_layer_size': {'values': [128, 256, 512]},
                'gamma': {'values': [0.5, 2, 4]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.01,
                                  'min': 0.0001},
                'momentum': {'distribution': 'uniform', 'max': 1, 'min': 0.7},
                'optimizer': {'values': ['adam', 'sgd']},
                'train_batch_size': {'distribution': 'q_log_uniform_values',
                                     'max': 128,
                                     'min': 32,
                                     'q': 1},
                'val_batch_size': {'distribution': 'q_log_uniform_values',
                                   'max': 256,
                         

In [ ]:
sweep_id = wandb.sweep(sweep_config, project = "fmri-dementia-sweep-six")

Create sweep with ID: aytv2g68
Sweep URL: https://wandb.ai/masadeghi6/fmri-dementia-sweep-six/sweeps/aytv2g68


# Training procedure

Notable resources used:

Focal loss source: https://github.com/mathiaszinnen/focal_loss_torch

ROC-AUC score: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score

In [ ]:
class_weights = class_weight.compute_class_weight('balanced', classes = labels['label'].unique(), y = labels['label'].values)
class_weights = torch.FloatTensor(class_weights).to(device)

In [ ]:
def train(config = None):

  with wandb.init(config = config):

    config = wandb.config

    train_data_loader = build_train_loader(config.train_batch_size)
    val_data_loader = build_val_loader(config.val_batch_size)

    model = build_model(config.architecture, config.fc_layer_size, config.dropout)
    criterion = FocalLoss(gamma = config.gamma, weights = class_weights)
    optimizer = build_optimizer(model, config.optimizer, config.learning_rate, config.momentum)

    for epoch in range(config.epochs):
      train_loss, val_loss, train_accuracy, val_accuracy, train_f1, val_f1, train_auc, val_auc = train_epoch(model, train_data_loader, val_data_loader, criterion, optimizer)
      wandb.log({"train_loss": train_loss, "val_loss": val_loss,
                 "train_accuracy": train_accuracy, "val_accuracy": val_accuracy,
                 "train_f1": train_f1, "val_f1": val_f1,
                 "train_auc": train_auc, "val_auc": val_auc,
                 "epoch": epoch})

# Create dataloaders

In [ ]:
class FMRILoader(Dataset):
    """
    fMRI Dataset Loader

    Args:
        path (str):
            Path to the CSV file

        labels (dict):
            Labels dictionnary if not provided by the CSV file

            AD: Alzheimer's disease
            FTD: Frontotemporal dementia
            HC: Healthy control
            MCI: Mild cognitive impairment
    """

    def __init__(self, path:str, labels:dict = {1:'AD', 2:'FTD', 3:'HC', 4:'MCI'}):
        self.folder = os.path.dirname(path)
        self.df = pd.read_csv(path)
        idname = dict()
        if 'class_name' in self.df:
            for i, row in self.df.iterrows():
                idname[row['label']] = row['class_name']
        else:
            idname = labels
        self.idclass = dict()
        for i, id in enumerate(set(idname)):
            self.idclass[id] = i
        self.classes = []
        for id in self.idclass:
            self.classes.append(idname[id])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        """
        Returns:
            A multichannel signal tensor and class id.
        """
        matrix_path = self.df['matrix'][idx]
        id = self.df['label'][idx]
        data = pd.read_csv(self.folder + '/scaled_data_dir/' + matrix_path, header = None).to_numpy()
        data = np.stack((data,)*3, axis = 0)
        data = data[:3, :200, :140]

        tensor = torch.tensor(data, dtype = torch.float32)

        return tensor, self.idclass[id]

In [ ]:
train_dataset = FMRILoader("/content/gdrive/MyDrive/Coding projects/resting_fmri_dementia/labels_train.csv")
val_dataset = FMRILoader("/content/gdrive/MyDrive/Coding projects/resting_fmri_dementia/labels_val.csv")

def build_train_loader(batch_size):

  train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

  return train_loader

def build_val_loader(batch_size):

  val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False)

  return val_loader

# Create model

In [ ]:
# model_architectures = list(sweep_config['parameters']['architecture'].items())[0][1]
# model_architectures = model_architectures if isinstance(model_architectures, list) else [model_architectures]

# models = {}
# #
# for net in model_architectures:
#   model = timm.create_model(net,
#                             pretrained = True,
#                             num_classes = 4,
#                             global_pool = 'catavgmax')
#   models[net] = model

In [ ]:
def build_model(architecture, fc_layer_size, dropout):

  model = timm.create_model(architecture,
                            pretrained = True,
                            num_classes = 4,
                            global_pool = 'catavgmax')

  num_in_features = model.get_classifier().in_features

  custom_head = nn.Sequential(
    nn.BatchNorm1d(num_in_features),
    nn.Linear(in_features = num_in_features, out_features = fc_layer_size, bias = False),
    nn.ReLU(inplace=True),
    nn.BatchNorm1d(fc_layer_size),
    nn.Dropout(dropout),
    nn.Linear(in_features = fc_layer_size, out_features = 4),
    nn.Softmax(dim = -1)
  )

  if 'resnet' in architecture:
    model.fc = custom_head

  return model.to(device)

# Choose optimizer

In [ ]:
def build_optimizer(model, optimizer, learning_rate, momentum):
  if optimizer == "sgd":
    optimizer = torch.optim.SGD(model.parameters(),
                          lr = learning_rate, momentum = momentum)
  elif optimizer == "adam":
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr = learning_rate)

  return optimizer

# Training loop

In [ ]:
def train_epoch(model, train_data_loader, val_data_loader, criterion, optimizer):

  for phase in ['train', 'validation']:
      if phase == 'train':
          model.train()
          dataset = train_dataset
          dataloader = train_data_loader
      else:
          model.eval()
          dataset = val_dataset
          dataloader = val_data_loader

      running_loss = 0.0
      running_outputs = torch.empty(0).to(device)
      running_labels = torch.empty(0).to(device)
      running_preds = torch.empty(0).to(device)

      for inputs, targets in dataloader:
          inputs, targets = inputs.to(device), targets.to(device)

          outputs = model(inputs)
          loss = criterion(outputs, targets)

          if phase == 'train':
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()

          preds = torch.argmax(outputs, axis = -1)
          running_preds = torch.cat([running_preds, preds])

          running_loss += loss.item() * inputs.size(0)
          running_outputs = torch.cat([running_outputs, outputs])
          running_labels = torch.cat([running_labels, targets])

      running_labels = running_labels.cpu()
      running_preds = running_preds.cpu()
      running_outputs = running_outputs.detach().cpu()

      if phase == 'train':
        train_loss = running_loss / len(dataset)
        train_accuracy = balanced_accuracy_score(running_labels, running_preds)
        train_f1 = f1_score(running_labels, running_preds, average = 'weighted')
        train_auc = roc_auc_score(running_labels, running_outputs, multi_class = "ovr", average = "weighted")

        # print('train loss: {:.4f}, accuracy: {:.4f}, f1: {:.4f}, auc: {:.4f}'.format(train_accuracy, train_loss, train_f1, train_auc))

      else:
        val_loss = running_loss / len(dataset)
        val_accuracy = balanced_accuracy_score(running_labels, running_preds)
        val_f1 = f1_score(running_labels, running_preds, average = 'weighted')
        val_auc = roc_auc_score(running_labels, running_outputs, multi_class = "ovr", average = "weighted")

        # print('validation loss: {:.4f}, accuracy: {:.4f}, f1: {:.4f}, auc: {:.4f}'.format(val_accuracy, val_loss, val_f1, val_auc))

  return train_loss, val_loss, train_accuracy, val_accuracy, train_f1, val_f1, train_auc, val_auc

# Initiate sweep

In [ ]:
wandb.agent(sweep_id, train, count = 10)

wandb: Agent Starting Run: 3a8jgkhc with config:
wandb: 	architecture: resnet18
wandb: 	dropout: 0
wandb: 	epochs: 8
wandb: 	fc_layer_size: 512
wandb: 	gamma: 0.5
wandb: 	learning_rate: 0.00434004880540479
wandb: 	momentum: 0.8992448972355611
wandb: 	optimizer: adam
wandb: 	train_batch_size: 122
wandb: 	val_batch_size: 230
wandb: Currently logged in as: masadeghi6. Use `wandb login --relogin` to force relogin


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


Run 3a8jgkhc errored: AssertionError(ValueError('The predictions values should be between 0 and 1,                 make sure to pass the values to sigmoid for binary                 classification or softmax for multi-class classification'))
wandb: ERROR Run 3a8jgkhc errored: AssertionError(ValueError('The predictions values should be between 0 and 1,                 make sure to pass the values to sigmoid for binary                 classification or softmax for multi-class classification'))
wandb: Agent Starting Run: fmsg3fbb with config:
wandb: 	architecture: resnet18
wandb: 	dropout: 0.6
wandb: 	epochs: 8
wandb: 	fc_layer_size: 512
wandb: 	gamma: 4
wandb: 	learning_rate: 0.009643949308079186
wandb: 	momentum: 0.7908350291535264
wandb: 	optimizer: sgd
wandb: 	train_batch_size: 87
wandb: 	val_batch_size: 88
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


epoch,▁▂▃▄▅▆▇█
train_accuracy,▁▃▄▅▆▇██
train_auc,▁▄▅▆▇▇██
train_f1,▁▃▄▅▆▇▇█
train_loss,█▅▃▂▂▂▁▁
val_accuracy,▁▂▄▇▆▆█▆
val_auc,▁▄▅▇▇▇█▇
val_f1,▁▄▆█▇██▇
val_loss,█▅▆▆▂▆▁▄
epoch,7
train_accuracy,0.81837


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 62y2mizp with config:
wandb: 	architecture: resnet18
wandb: 	dropout: 0.4
wandb: 	epochs: 8
wandb: 	fc_layer_size: 256
wandb: 	gamma: 4
wandb: 	learning_rate: 0.0006041409578696525
wandb: 	momentum: 0.8668885783762565
wandb: 	optimizer: adam
wandb: 	train_batch_size: 78
wandb: 	val_batch_size: 48
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

wandb: Agent Starting Run: hwbqgftc with config:
wandb: 	architecture: resnet18
wandb: 	dropout: 0
wandb: 	epochs: 8
wandb: 	fc_layer_size: 256
wandb: 	gamma: 2
wandb: 	learning_rate: 0.0034077622827766133
wandb: 	momentum: 0.9311392524793592
wandb: 	optimizer: adam
wandb: 	train_batch_size: 41
wandb: 	val_batch_size: 162


epoch,▁▂▃▄▅▆▇█
train_accuracy,▁▃▄▆▇█▆▇
train_auc,▁▄▅▅▇▇▇█
train_f1,▁▁▄▆████
train_loss,█▃▃▂▁▁▁▁
val_accuracy,▁▃▄█▅▆▇▇
val_auc,▁▁▂▃▄▃▁█
val_f1,▁▅▆▇▇▇█▅
val_loss,█▃▂▂▁▁▃▁
epoch,7
train_accuracy,0.53664


wandb: Agent Starting Run: 3xb48p5m with config:
wandb: 	architecture: resnet18
wandb: 	dropout: 0.4
wandb: 	epochs: 8
wandb: 	fc_layer_size: 256
wandb: 	gamma: 0.5
wandb: 	learning_rate: 0.00926410893735149
wandb: 	momentum: 0.8933300941578308
wandb: 	optimizer: sgd
wandb: 	train_batch_size: 35
wandb: 	val_batch_size: 62


epoch,▁
train_accuracy,▁
train_auc,▁
train_f1,▁
train_loss,▁
val_accuracy,▁
val_auc,▁
val_f1,▁
val_loss,▁
epoch,0
train_accuracy,0.4387


Run 3xb48p5m errored: AssertionError(ValueError('The predictions values should be between 0 and 1,                 make sure to pass the values to sigmoid for binary                 classification or softmax for multi-class classification'))
wandb: ERROR Run 3xb48p5m errored: AssertionError(ValueError('The predictions values should be between 0 and 1,                 make sure to pass the values to sigmoid for binary                 classification or softmax for multi-class classification'))
